# Crawling et utilisation de l'API distance24.org

INFMDI721 - TP - 16/10/2019

Consigne:

- Trouver les 10 villes de France les + peuplées (e.g https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peuplées)
- Pour chaque ville, trouver sa distance avec les autres. Par exemple, avec google maps api (nécessite de s'authentifier / créer un token) avec https://github.com/googlemaps/google-maps-services-python (cf. doc token dans le readme) ou avec https://fr.distance24.org/ (pas besoin de s'authentifier)
- Trouver les villes les plus proches


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Récupération de la liste des villes

In [4]:
page = "https://fr.wikipedia.org/wiki/Liste_des_communes_de_France_les_plus_peupl%C3%A9es"
response = requests.get(page)
soup = BeautifulSoup(response.text)

In [5]:
rows = soup.find("table").find_all("tr")

In [6]:
cities = []
for row in rows:
    city = row.find_all("td")
    if len(city) > 1:
        city = city[1].find("a").text
        cities.append(city)
        
cities = cities[0:10]
print(cities)

['Paris', 'Marseille', 'Lyon', 'Toulouse', 'Nice', 'Nantes', 'Montpellier', 'Strasbourg', 'Bordeaux', 'Lille']


## Remplissage d'un dataframe à partir de l'API

In [48]:
df = pd.DataFrame(columns=cities)
df.head()

,Paris,Marseille,Lyon,Toulouse,Nice,Nantes,Montpellier,Strasbourg,Bordeaux,Lille


In [49]:
def distance(cityA, cityB):
    url = "https://fr.distance24.org/route.json?stops="+cityA+"|"+cityB
    res = requests.get(url).json()
    return res["distance"]

In [50]:
for cityA in cities:
    print(cityA)
    distances = {}
    for cityB in cities:
        distances[cityB] = distance(cityA, cityB)
    df = df.append(distances, ignore_index=True)

Paris
Marseille
Lyon
Toulouse
Nice
Nantes
Montpellier
Strasbourg
Bordeaux
Lille


In [51]:
df.index = cities
df.head(10)

,Paris,Marseille,Lyon,Toulouse,Nice,Nantes,Montpellier,Strasbourg,Bordeaux,Lille
Paris,0,661,392,589,688,343,596,397,500,204
Marseille,661,0,278,319,159,696,126,617,506,835
Lyon,392,278,0,360,300,516,251,384,436,558
Toulouse,589,319,360,0,469,466,196,737,212,792
Nice,688,159,300,469,0,792,273,545,638,835
Nantes,343,696,516,466,792,0,584,710,276,508
Montpellier,596,126,251,196,273,584,0,629,381,784
Strasbourg,397,617,384,737,545,710,629,0,760,408
Bordeaux,500,506,436,212,638,276,381,760,0,700
Lille,204,835,558,792,835,508,784,408,700,0
